In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import time

Data

In [ ]:
Feature = ['Day','Time','NUM','AIR_STAT']
Num_Feature = len(Feature)
df = pd.read_excel('Data_train.xlsx',sheet_name='DATA2')
# AIR_STAT 0=off, 1=open
# TES_STAT 0=off, 1=use, 2=charge

X= df.drop('TES_STAT',axis = 1) # ตัด TES_STAT ออก
Y = df.TES_STAT

#-----------Data Train/Data Test----------------------
sc = StandardScaler()
x_sc = sc.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(x_sc,Y,test_size=0.3)


In [ ]:
#Tester block data
#X.head()
#Y.head()
#x_train.shape, x_test.shape
#y_train.shape, y_test.shape

Modek AI LSTM

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
model = Sequential([
    Dense(16, activation='relu',input_shape= (Num_Feature,)),
    Dropout(0.2),
    Dense(3, activation = 'softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

# จับเวลา train
start = time.time()
history = model.fit(x_train,y_train,
    validation_data=(x_test, y_test),
    epochs = 500,
    verbose = 1,
    batch_size = 32,
    validation_split = 0.3,
    callbacks=[es]
)
end = time.time()

#การวัดค่า score
score = model.evaluate(x_test,y_test,verbose = 0)

In [ ]:
print('Test loss : ',score[0])
print('Test accuracy AI:', score[1],' or ', score[1]*100, '%')
#นำ test เข้าทดสอบ
y_pred = model.predict(x_test)
#print(y_pred)
#print(x_test)
#print(y_pred)

Plot graph Loss

In [ ]:
df_history = pd.DataFrame.from_dict(history.history)
history.params
history.history.keys()
#--------plot-------
df_history['loss'].plot(style='b--',label='Trainning')
df_history['val_loss'].plot(style='r-',label = 'Test')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss')
plt.show()

Save model

In [ ]:
print("\n--- Saving Model for MATLAB/Simulink ---")
model.save("Model1_2.h5")

# ----------------------- บันทึก Scaler -----------------------
joblib.dump(sc, 'AI_V1_2.save')
print("✅ โมเดลและ Scaler ถูกบันทึกเรียบร้อยแล้ว")